In [5]:
import tensorflow as tf
import os

In [9]:
W = tf.Variable(tf.zeros([4, 3]), name='weights')
b = tf.Variable(tf.zeros([3]), name='bias')


# former inference is now used for combining inputs
def combine_inputs(X):
    return tf.matmul(X, W) + b


# define the training loop operations
def inference(X):
    # computer inference model over data X and return the result
    return tf.nn.softmax(combine_inputs(X))
    

def loss(X, Y):
    # computer loss over training data X and expected outputs Y
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(combine_inputs(X), Y))


def read_csv(batch_size, filename, record_defaults):
    # https://www.tensorflow.org/how_tos/reading_data/
    filename_queue = tf.train.string_input_producer([os.path.join(os.getcwd(), filename)])
    
    reader = tf.TextLineReader(skip_header_lines=0)
    key, value = reader.read(filename_queue)
    
    # decode_csv will convert a Tensor from type string (the text line) in
    # a tuple of tensor columns with the specified defaults, which also,
    # sets the data type for each column
    decoded = tf.decode_csv(value, record_defaults=record_defaults)
    
    # batch actually reads the file and loads "batch_size" rows in a single tensor
    return tf.train.shuffle_batch(decoded, batch_size=batch_size,
                                  capacity=batch_size * 50, min_after_dequeue=batch_size)


def inputs():
    # https://archive.ics.uci.edu/ml/datasets/Iris
    sepal_length, sepal_width, petal_length, petal_width, label = \
        read_csv(100, "iris.data", [[0.0], [0.0], [0.0], [0.0], [""]])

    # convert class names to a 0 based class index
    label_number = tf.to_int32(tf.argmax(tf.to_int32(tf.pack([
                        tf.equal(label, ["Iris-setosa"]),
                        tf.equal(label, ["Iris-versicolor"]),
                        tf.equal(label, ["Iris-virginica"])
                    ])), 0))

    # pack all the features that we care about in a single matrix;
    # we then transpose to have a matrix with one example per row and one feature per column.
    features = tf.transpose(tf.pack([sepal_length, sepal_width, petal_length, petal_width])) 

    return features, label_number


def train(total_loss):
    # train / adjust model parameters according to computed total loss
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)


def evaluate(sess, X, Y):
    # evaluate the resulting trained model
    predicted = tf.cast(tf.argmax(inference(X), 1), tf.int32)
    print sess.run(tf.reduce_mean(tf.cast(tf.equal(predicted, Y), tf.float32)))


saver = tf.train.Saver()

# launch the graph in a session, setup boilerplate
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    
    X, Y = inputs()
    
    total_loss = loss(X, Y)
    train_op = train(total_loss)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    initial_step = 0 
    
    # verify if we don't have a checkpoint saved already
    #ckpt = tf.train.get_checkpoint_state(os.path.dirname("."))
    #if ckpt and ckpt.model_checkpoint_path:
    #    #Restores from checkpoint
    #    saver.restore(sess, ckpt.model_checkpoint_path)
    #    initial_step = int(ckpt.model_checkpoint_path.rsplit('-', 1)[1])
        
    training_steps = 1000
    for step in range(initial_step, training_steps):
        sess.run([train_op])
        
        #if step % 1000 == 0:
        #    saver.save(sess, 'my-model', global_step=step)
            
        # for debugging and learning purposes, see how the loss gets decremented through training steps.
        if step % 10 == 0:
            print "loss: ", sess.run([total_loss])
                
    evaluate(sess, X, Y)
    
    #saver.save(sess, 'my-model', global_step=training_steps)
    
    coord.request_stop()
    coord.join(threads)
    sess.close()

loss:  [1.0918864]
loss:  [1.0076835]
loss:  [0.96180207]
loss:  [0.92809641]
loss:  [0.8594355]
loss:  [0.82788664]
loss:  [0.81425363]
loss:  [0.78638428]
loss:  [0.73824936]
loss:  [0.75816405]
loss:  [0.70593011]
loss:  [0.65243173]
loss:  [0.68002439]
loss:  [0.65541506]
loss:  [0.63412756]
loss:  [0.62858719]
loss:  [0.61165625]
loss:  [0.58983636]
loss:  [0.6388666]
loss:  [0.56678194]
loss:  [0.54201567]
loss:  [0.56649756]
loss:  [0.60231709]
loss:  [0.5364992]
loss:  [0.59488243]
loss:  [0.54561806]
loss:  [0.50032568]
loss:  [0.53931218]
loss:  [0.55437028]
loss:  [0.5170095]
loss:  [0.53963232]
loss:  [0.51689804]
loss:  [0.50891566]
loss:  [0.48621696]
loss:  [0.49127173]
loss:  [0.47326669]
loss:  [0.52866369]
loss:  [0.47955713]
loss:  [0.49915311]
loss:  [0.5243125]
loss:  [0.48467645]
loss:  [0.44337234]
loss:  [0.44426566]
loss:  [0.48620296]
loss:  [0.45069274]
loss:  [0.50417536]
loss:  [0.4898037]
loss:  [0.43416661]
loss:  [0.49357334]
loss:  [0.45336398]
loss:  [